In [1]:
from PIL import Image
import pandas as pd
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
def carregar_imagem(arquivo):
    imagem = Image.open(arquivo)
    image = imagem.convert('RGB')

    return asarray(imagem)

In [3]:
def carregar_faces(diretorio_src):
    faces = list()
    for nome_arquivo in listdir(diretorio_src):
        path = diretorio_src + nome_arquivo
        try:
            faces.append(carregar_imagem(path))
        except:
            print("Erro ao carregar a imagem: ", path)
    return faces

In [4]:
def carregar_fotos(diretorio_src): # endereço das fotos de entrada
    X, y = list(), list()
    for subdir in listdir(diretorio_src):
        path = diretorio_src + subdir + '\\'
        if not isdir(path):
            continue
        faces = carregar_faces(path)
        labels = [subdir for _ in range(len(faces))]
        print(f'>Carregadas {len(faces)} faces da classe: {subdir}')
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [5]:
#carregando todas as fotos
treinoX, treinoY = carregar_fotos(diretorio_src='F:\\Bootcamp_DIO\\desafio_reconhecer_rostos\\validacao\\')

>Carregadas 41 faces da classe: Desconhecido
>Carregadas 42 faces da classe: MajuCoutinho
>Carregadas 42 faces da classe: SeuJorge
>Carregadas 42 faces da classe: WillSmith


In [6]:
print(treinoX.shape, treinoY.shape)

(167, 160, 160, 3) (167,)


In [7]:
modelo = load_model('..\\modelos\\facenet_keras.h5', compile=False)

In [8]:
modelo.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv2d_1a_3x3 (Conv2D)         (None, 79, 79, 32)   864         ['input_1[0][0]']                
                                                                                                  
 Conv2d_1a_3x3_BatchNorm (Batch  (None, 79, 79, 32)  96          ['Conv2d_1a_3x3[0][0]']          
 Normalization)                                                                                   
                                                                                

In [9]:
# gerar embedding
def get_embedding(modelo, face_pixels):
    # face_pixels = face_pixels.astype('float32')
    media, desvio_padrao = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - media) / desvio_padrao

    amostras = expand_dims(face_pixels, axis=0)
    yhat = modelo.predict(amostras)
    return yhat[0]

In [10]:
novoTreinoX = list()

In [11]:
for face_pixels in treinoX:
    embedding = get_embedding(modelo, face_pixels)
    novoTreinoX.append(embedding)

1/1 [==============================] - 0s 238ms/step


In [12]:
novotrainX = asarray(novoTreinoX)
novoTreinoX[0].shape

(128,)

In [13]:
df = pd.DataFrame(novoTreinoX)
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.868100,-2.596633,-0.189149,1.354571,-0.279782,-1.569395,-1.077242,0.879807,-0.046270,0.016018,...,-0.104218,-2.228687,1.208452,-0.768819,0.813632,-1.235054,-1.020071,-0.474822,0.056766,0.563398
1,0.091845,0.568946,-0.570795,1.397774,1.583693,-0.782292,-0.943915,-1.174724,1.553781,-2.666841,...,1.628370,-1.094370,0.499978,-0.337125,0.694377,-0.057151,1.678644,-1.608734,-0.049961,-0.801952
2,-1.490249,0.683534,-0.865304,0.725783,-0.238300,-0.031432,-1.832989,-0.749937,-0.255921,-2.465865,...,0.103148,-0.637263,0.721525,0.026797,1.728883,0.868392,0.640219,-0.351550,-0.387878,1.166220
3,-0.911458,0.850990,-0.066679,0.664383,-0.795028,1.665197,-0.520423,-1.052025,0.703121,-2.655503,...,0.615217,-0.328937,-1.112960,0.626143,1.934606,-1.686491,0.249525,-1.675047,1.036272,-1.084320
4,0.950458,1.292446,-0.750623,-0.362144,0.557248,-0.016750,-0.773281,1.498607,-0.510444,1.009430,...,1.243496,-1.501625,-1.141505,-0.931672,-0.150145,2.203952,0.708816,-0.645506,0.520323,0.040765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0.118480,-1.747889,0.895346,-0.708635,0.064910,-0.659604,-1.272586,-0.380964,-0.086799,0.597620,...,-1.359865,0.275491,0.268442,0.062873,0.259845,-1.369199,-0.785729,-1.856425,-1.230433,-0.069118
163,0.541478,-0.569726,0.048284,-0.991950,-0.779280,0.514487,-0.260818,-0.142966,0.446527,0.158517,...,-0.181633,-0.055334,-0.004297,0.428483,1.347580,-1.136228,0.229152,-1.736536,-0.591302,0.489791
164,1.441094,-0.693641,-0.758835,-0.431055,-0.539901,0.673711,-0.254791,-0.624513,0.449178,-0.153305,...,0.392870,-0.456633,0.308733,0.272660,1.754868,-1.393099,-0.016938,-1.981347,-0.522422,0.339097
165,0.896200,-0.652182,0.376279,-0.790490,-0.789988,0.448904,-0.663589,-0.257222,0.692736,0.422864,...,-0.720681,-0.730942,-0.476848,-0.163519,1.309300,-1.177787,0.106684,-2.152738,-0.580391,0.134769


In [14]:
df['alvo'] = treinoY
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,alvo
0,-0.868100,-2.596633,-0.189149,1.354571,-0.279782,-1.569395,-1.077242,0.879807,-0.046270,0.016018,...,-2.228687,1.208452,-0.768819,0.813632,-1.235054,-1.020071,-0.474822,0.056766,0.563398,Desconhecido
1,0.091845,0.568946,-0.570795,1.397774,1.583693,-0.782292,-0.943915,-1.174724,1.553781,-2.666841,...,-1.094370,0.499978,-0.337125,0.694377,-0.057151,1.678644,-1.608734,-0.049961,-0.801952,Desconhecido
2,-1.490249,0.683534,-0.865304,0.725783,-0.238300,-0.031432,-1.832989,-0.749937,-0.255921,-2.465865,...,-0.637263,0.721525,0.026797,1.728883,0.868392,0.640219,-0.351550,-0.387878,1.166220,Desconhecido
3,-0.911458,0.850990,-0.066679,0.664383,-0.795028,1.665197,-0.520423,-1.052025,0.703121,-2.655503,...,-0.328937,-1.112960,0.626143,1.934606,-1.686491,0.249525,-1.675047,1.036272,-1.084320,Desconhecido
4,0.950458,1.292446,-0.750623,-0.362144,0.557248,-0.016750,-0.773281,1.498607,-0.510444,1.009430,...,-1.501625,-1.141505,-0.931672,-0.150145,2.203952,0.708816,-0.645506,0.520323,0.040765,Desconhecido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0.118480,-1.747889,0.895346,-0.708635,0.064910,-0.659604,-1.272586,-0.380964,-0.086799,0.597620,...,0.275491,0.268442,0.062873,0.259845,-1.369199,-0.785729,-1.856425,-1.230433,-0.069118,WillSmith
163,0.541478,-0.569726,0.048284,-0.991950,-0.779280,0.514487,-0.260818,-0.142966,0.446527,0.158517,...,-0.055334,-0.004297,0.428483,1.347580,-1.136228,0.229152,-1.736536,-0.591302,0.489791,WillSmith
164,1.441094,-0.693641,-0.758835,-0.431055,-0.539901,0.673711,-0.254791,-0.624513,0.449178,-0.153305,...,-0.456633,0.308733,0.272660,1.754868,-1.393099,-0.016938,-1.981347,-0.522422,0.339097,WillSmith
165,0.896200,-0.652182,0.376279,-0.790490,-0.789988,0.448904,-0.663589,-0.257222,0.692736,0.422864,...,-0.730942,-0.476848,-0.163519,1.309300,-1.177787,0.106684,-2.152738,-0.580391,0.134769,WillSmith


In [15]:
df.to_csv('..\\dados\\faces_validacao.csv', index=False)